<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Обзор данных</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#EDA" data-toc-modified-id="EDA-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>EDA</a></span></li></ul></div>

# Анализ работы в доставке
**Цель** - сделать выводы и рекомендации для себя и для бизнеса по собранным данным.

**Задачи:**
* [X] предобработать данные в обоих таблицах;
* [ ] построить карту заказов по таблице df1(столбец `respawn`), выяснить активные районы/дома/улицы, сделать выводы и рекомендации;
* [ ] сгруппировать данные по дню с подсчетом количества заказов в **df1** для объединения с **df2**;
* [ ] найти инсайты в данных.

# Описание данных
**df1** - расширенная информация по каждому заказу с 24.07.2021г. по 12.12.2021г.
* `datetime` - дата заказа;
* `user_id` - уникальный id пользователя;
* `nickname` - имя пользователя;
* `respawn` - адрес заказа;
* `revenue` - стоимость заказа;
* `device_ios_or_android` - телефон на платформе ios("0") или android("1");
* `courier` - кодовое обозначения курьера;
* `restaurant` - место выдачи заказа.

**df2** - упрощенная информация с количеством заказов и итоговой суммой в день с 15.12.2021г. по 09.10.2022г.
* `date` - отчётный день;
* `source_income` - источник дохода(bk - burger king, dc - delivery club, lenta, yandex_eda);
* `orders` - выполнено заказов;
* `slot_duration` - продолжительность слота;
* `restaurant` - место выдачи заказа;
* `income` - выручка за день.

## Обзор данных

In [1]:
# импорт библиотек
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

In [2]:
# прочитаем и сохраним данные
df1 = pd.read_csv('/datasets/betat.csv')
df2 = pd.read_csv('/datasets/car_income.csv')

In [3]:
# посмотрим на 1 таблицу
df1.head()

,datetime,user_id,nickname,respawn,revenue,device_ios_or_android,courier,restaurant
0,24.07.2021 10:29:00,79878736010,Владимир,"Оренбург, ул. Постникова, д. 9б,_,_,_",693.00,1,color,Sovet
1,24.07.2021 10:33:00,79033667992,Ольга,"Оренбург, ул. Чкалова, д. 28, кв. 219, эт. 12,...",737.00,0,color,Sovet
2,24.07.2021 11:05:00,79878418091,Виктория,"Оренбург, ул. Томилинская, д. 235,_,_,_",638.00,unknown,color,Sovet
3,24.07.2021 12:19:00,79228100614,unknown,"Оренбург, ул. Деповская, д. 96,_,_,_",514.99,0,color,Sovet
4,24.07.2021 12:25:00,79010857083,Роман,"Оренбург, пр-кт Победы, д. 13д,_,_,_",601.00,unknown,color,Sovet


In [4]:
# посмотрим на 2 таблицу
df2.head()

,date,source_income,orders,slot_duration,restaurant,income
0,15.12.2021,bk,16,12.0,Sovet,NaN
1,16.12.2021,bk,13,10.0,Sovet,NaN
2,17.12.2021,bk,13,12.0,Sovet,NaN
3,18.12.2021,bk,21,11.0,Sovet,NaN
4,19.12.2021,bk,13,12.0,Sovet,NaN


Видим данные, их нужно будет подготовить и объединить. Объединять будем по дате. Изучим каждый датасет в отдельности и поймём с чем нужно будет работать.

**df1**

In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1273 entries, 0 to 1272
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   datetime               1273 non-null   object 
 1   user_id                1273 non-null   int64  
 2   nickname               1273 non-null   object 
 3   respawn                1273 non-null   object 
 4   revenue                1273 non-null   float64
 5   device_ios_or_android  1273 non-null   object 
 6   courier                1273 non-null   object 
 7   restaurant             1273 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 79.7+ KB


**Общая информация:**
* строк - 1273;
* столбцов - 8.

Названия столбцов в snake_case стиле. В доработке не нуждаются.

**Типы данных:**
* float - 1;
* int - 1;
* object - 6.

Необходимо сменить тип данных в столбце `datetime` на datetime. Проработать момент с `device_ios_or_android` значение "unknown" говорит нам о том, что заказ был сделал из мобильного приложения "delivery club".

In [6]:
df1.isna().mean()

datetime                 0.0
user_id                  0.0
nickname                 0.0
respawn                  0.0
revenue                  0.0
device_ios_or_android    0.0
courier                  0.0
restaurant               0.0
dtype: float64

Доля пропусков 0.

In [7]:
df1.duplicated().sum()

0

Явные дубликаты отсутствуют.

In [43]:
df1.nickname.unique()

array(['Владимир', 'Ольга', 'Виктория', 'unknown', 'Роман', 'Алина',
       'Дмитрий', 'Наталья', 'Екатерина', 'Дарья', 'Светлана', 'Арина',
       'Антон', 'Илья', 'Даниил', 'Гузель', 'Кирилл', 'Яна', 'Алена',
       'Джордж', 'Вячеслав', 'Алексей', 'Егор', 'Павел', 'Рузанна',
       'Никита', 'Александра', 'Лилия', 'Анастасия', 'Игорь', 'Олеся',
       'Рустам', 'Александр', 'Маргарита', 'Артем', 'Максим', 'Ирина',
       'Николай', 'Жанна', 'Юлия', 'Кристина', 'Ринат', 'Татьяна',
       'Евгений', 'Руслан', 'Виктор', 'Елена', 'Мария', 'Кумар', 'Сергей',
       'Венера', 'Юлиана', 'Ксения', 'Вероника', 'Валерия', 'Нина',
       'Вадим', 'Марина', 'Карина', 'Ерлан', 'Айгуль', 'Эльнора',
       'Альбек', 'Регина', 'Дина', 'Виолетта', 'Денис', 'Лунара',
       'Рузалия', 'Лиана', 'Азат', 'Ляззат', 'Саула', 'Вильдан',
       'Елизавета', 'Седа', 'Инга', 'Аманат', 'Валерий', 'Михаил',
       'Герман', 'Софья', 'Лидия', 'Розалия', 'Матвей', 'Вера', 'Инна',
       'Анна', 'Fernando', 'Рости

В столбце `nickname` присутствуют неявные дубликаты.

In [9]:
df1.user_id.value_counts()[:5]

79033903911    5
79870428340    5
79228347146    5
79878845157    4
79242997725    4
Name: user_id, dtype: int64

Есть повторы, которые говорят нам, что клиент заказывал не один раз, это хорошо для бизнеса.

In [10]:
df1.respawn.value_counts()[:5]

Оренбург, пр-кт Победы, д. 13Д,_,_,_                      6
Оренбург, ул. Терешковой, д. 10/5, кв. 14, эт. 4, п. 1    5
Оренбург, ул. Полигонная, д. 1/1, кв. 46, эт. 2, п. 2     5
Оренбург, ул. Постникова, д. 9Б,_,_,_                     5
Оренбург, ул. Ткачева, д. 85, кв. 34, эт. 9, п. 1         5
Name: respawn, dtype: int64

Повторы говорят про популярные адреса откуда поступают заказы.

In [11]:
df1.revenue.value_counts().head(5)

699.0     33
1099.0    21
649.0     14
799.0     11
600.0     11
Name: revenue, dtype: int64

Прослеживаются популярные суммы заказов, может быть, сумма связана с бесплатной доставкой. В таком случаем за счёт этого можно поднять средний чек.

In [12]:
df1.courier.value_counts()

red      1236
color      37
Name: courier, dtype: int64

Есть данные по работе двух курьеров. Записей заказов 'red' намного больше.

In [13]:
df1.restaurant.value_counts()

Sovet     1002
Teresh     179
Salm        56
Gagar       36
Name: restaurant, dtype: int64

Доставки осуществлялись c 4 ресторанов Burger King. Больше всего заказов вывезено с "Sovet".

**df2**

In [14]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           174 non-null    object 
 1   source_income  174 non-null    object 
 2   orders         174 non-null    int64  
 3   slot_duration  174 non-null    float64
 4   restaurant     67 non-null     object 
 5   income         113 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 8.3+ KB


**Общая информация:**
* строк - 174;
* столбцов - 6.

Названия столбцов в snake_case стиле. В доработке не нуждаются.

**Типы данных:**
* float - 2;
* int - 1;
* object - 3.

Необходимо сменить тип данных в столбце `date` на datetime. Проработать момент с `slot_duration`.

In [15]:
df2.isna().mean()

date             0.000000
source_income    0.000000
orders           0.000000
slot_duration    0.000000
restaurant       0.614943
income           0.350575
dtype: float64

**Пропуски:**
* restaurant - 61%;
* income - 35%.

Выясним с чем связаны пропуски.

In [16]:
df2.restaurant.unique()

array(['Sovet', 'Teresh', 'Gagar', 'tk113', 'tk248', 'tk240', nan],
      dtype=object)

**Точки старта burger king:**
* 'Sovet';
* 'Teresh'; 
* 'Gagar'.

**Точки старта lenta:**
* 'tk113';
* tk248';
* 'tk240'.

**Точки старта delivery club:**
* NaN.

Пропуски связаны с тем, что старт слота в delivery club свободный и не привязан к ресторану, как в burger king или lenta.

In [17]:
df2.income.unique()

array([  nan,  810.,  760., 2820., 2550., 2730., 1006., 1411., 1625.,
       1905., 1018., 1617.,  619.,  613., 2292., 2400., 1581., 1224.,
        412., 1750.,  205., 2742., 1342., 1656., 2487., 1696., 1622.,
       1321., 3603., 1790., 1619., 1503., 1356., 1320., 1021., 1547.,
       1803., 1705., 1180., 1226., 1778., 1808., 1116., 1694.,  927.,
        136., 2754., 1478., 2150., 1521., 1773., 1166., 2088., 1339.,
       2084., 3299., 1375., 2107., 3270., 1386., 1460., 1193., 1553.,
        925., 1421., 2141., 3093., 2310., 1988., 1663., 2402., 1745.,
       2115., 1574., 1281., 2757., 2203., 1664., 1082., 2600., 3557.,
       1586., 1815., 1487., 2132., 1873., 2740., 2689.,  261., 2934.,
       3690., 2215., 2123., 2492.,  733., 2121., 1753., 3890., 1216.,
       2287., 3541., 2621., 3294., 1220., 2186., 2134., 2426., 1330.,
       4776., 2144., 1154., 1526.,  174.])

Пропуски появились из-за burger king, для их заполнения необходимо число заказов(`orders`) перемножить на 150(стоимость одного заказа).

In [18]:
df2.duplicated().sum()

0

Явных дубликатов нет.

In [19]:
df2.source_income.value_counts()

dc       107
bk        61
lenta      6
Name: source_income, dtype: int64

In [20]:
df2.slot_duration.value_counts()

12.0    24
4.0     24
8.0     21
6.0     18
9.0     17
7.0     17
5.0     16
11.0    12
10.0     8
13.0     4
1.0      2
9.5      2
14.0     2
3.0      2
13.5     1
7.5      1
2.0      1
12.5     1
0.0      1
Name: slot_duration, dtype: int64

In [21]:
df2.restaurant.value_counts()

Sovet     37
Gagar     17
Teresh     7
tk248      3
tk240      2
tk113      1
Name: restaurant, dtype: int64

Неявных дубликатов нет.

### Вывод

**df1**

**Общая информация:**
* строк - 1273;
* столбцов - 8.

Названия столбцов в snake_case стиле. В доработке не нуждаются.

**Типы данных:**
* float - 1;
* int - 1;
* object - 6.

Необходимо сменить тип данных в столбце `datetime` на datetime. Проработать момент с `device_ios_or_android` значение "unknown" говорит нам о том, что заказ был сделал из мобильного приложения "delivery club".

Пропуски и явные дубликаты в данных отсутствуют.

В столбце `nickname` присутствуют неявные дубликаты.

**df2**

**Общая информация:**
* строк - 174;
* столбцов - 6.

Названия столбцов в snake_case стиле. В доработке не нуждаются.

**Типы данных:**
* float - 2;
* int - 1;
* object - 3.

Необходимо сменить тип данных в столбце `date` на datetime. Проработать момент с `slot_duration`.

**Пропуски:**
* restaurant - 61%;
* income - 35%.

Пропуски в `restaurant` связаны с тем, что старт слота в delivery club свободный и не привязан к ресторану, как в burger king или lenta.

Пропуски в `income` появились из-за burger king(bk), для их заполнения необходимо число заказов(`orders`) перемножить на 150(стоимость одного заказа).

Явных/неявных дубликатов нет.

## Предобработка данных

**df1**

In [22]:
# взглянем на данные
df1.head()

,datetime,user_id,nickname,respawn,revenue,device_ios_or_android,courier,restaurant
0,24.07.2021 10:29:00,79878736010,Владимир,"Оренбург, ул. Постникова, д. 9б,_,_,_",693.00,1,color,Sovet
1,24.07.2021 10:33:00,79033667992,Ольга,"Оренбург, ул. Чкалова, д. 28, кв. 219, эт. 12,...",737.00,0,color,Sovet
2,24.07.2021 11:05:00,79878418091,Виктория,"Оренбург, ул. Томилинская, д. 235,_,_,_",638.00,unknown,color,Sovet
3,24.07.2021 12:19:00,79228100614,unknown,"Оренбург, ул. Деповская, д. 96,_,_,_",514.99,0,color,Sovet
4,24.07.2021 12:25:00,79010857083,Роман,"Оренбург, пр-кт Победы, д. 13д,_,_,_",601.00,unknown,color,Sovet


In [46]:
# сменим тип данных в столбце "datetime"
df1['datetime'] = pd.to_datetime(df1['datetime'], format='%d.%m.%Y %H:%M:%S')
# проверим результат
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1273 entries, 0 to 1272
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   datetime               1273 non-null   datetime64[ns]
 1   user_id                1273 non-null   int64         
 2   nickname               1273 non-null   object        
 3   respawn                1273 non-null   object        
 4   revenue                1273 non-null   float64       
 5   device_ios_or_android  1273 non-null   object        
 6   courier                1273 non-null   object        
 7   restaurant             1273 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 79.7+ KB


Тип данных изменён.

In [24]:
# займёмся столбцом device_ios_or_android
df1.device_ios_or_android.value_counts()

unknown     712
1           374
0           186
unknown"      1
Name: device_ios_or_android, dtype: int64

Видим неявный дубликат `unknown"`.

In [25]:
# уберем дубликат
df1['device_ios_or_android'] = df1['device_ios_or_android'].replace('unknown"', 'unknown')
df1.device_ios_or_android.value_counts()

unknown    713
1          374
0          186
Name: device_ios_or_android, dtype: int64

Дубликат убран. Займёмся столбцом `nickname`.

In [26]:
# уникальных имен до изменений
len(df1.nickname.unique())

268

In [47]:
# список уникальных имен
df1.nickname.unique()

array(['Владимир', 'Ольга', 'Виктория', 'unknown', 'Роман', 'Алина',
       'Дмитрий', 'Наталья', 'Екатерина', 'Дарья', 'Светлана', 'Арина',
       'Антон', 'Илья', 'Даниил', 'Гузель', 'Кирилл', 'Яна', 'Алена',
       'Джордж', 'Вячеслав', 'Алексей', 'Егор', 'Павел', 'Рузанна',
       'Никита', 'Александра', 'Лилия', 'Анастасия', 'Игорь', 'Олеся',
       'Рустам', 'Александр', 'Маргарита', 'Артем', 'Максим', 'Ирина',
       'Николай', 'Жанна', 'Юлия', 'Кристина', 'Ринат', 'Татьяна',
       'Евгений', 'Руслан', 'Виктор', 'Елена', 'Мария', 'Кумар', 'Сергей',
       'Венера', 'Юлиана', 'Ксения', 'Вероника', 'Валерия', 'Нина',
       'Вадим', 'Марина', 'Карина', 'Ерлан', 'Айгуль', 'Эльнора',
       'Альбек', 'Регина', 'Дина', 'Виолетта', 'Денис', 'Лунара',
       'Рузалия', 'Лиана', 'Азат', 'Ляззат', 'Саула', 'Вильдан',
       'Елизавета', 'Седа', 'Инга', 'Аманат', 'Валерий', 'Михаил',
       'Герман', 'Софья', 'Лидия', 'Розалия', 'Матвей', 'Вера', 'Инна',
       'Анна', 'Fernando', 'Рости

In [28]:
df1['nickname'] = df1['nickname'].replace({'Паша': 'Павел', 
                                           'Julia': 'Юлия',
                                           'Юличка': 'Юлия',
                                           'Оля': 'Ольга',
                                           'Вика': 'Виктория',
                                           'Катя': 'Екатерина',
                                           'Катенька': 'Екатерина',
                                           'Катерина': 'Екатерина',
                                           'Саша': 'Александр',
                                           'Влада': 'Владислава',
                                           'Влад': 'Владислав',
                                           'Вован': 'Владимир',
                                           'Лиза': 'Елизавета',
                                           'Катюня': 'Екатерина',
                                           'Ekaterina': 'Екатерина',
                                           'Slava': 'Вячеслав',
                                           'Слава': 'Вячеслав',
                                           'Ксюша': 'Ксения',
                                           'Ксю': 'Ксения',
                                           'Валера': 'Валерий',
                                           'Леся': 'Олеся',
                                           'Дима': 'Дмитрий',
                                           'Лена': 'Елена',
                                           'Настя': 'Анастасия',
                                           'Виола': 'Виолетта',
                                           '123': 'unknown',
                                           'Дорджи': 'Джордж',
                                           'Джорджи': 'Джордж',
                                           'Даша': 'Дарья',
                                           'Ника': 'Вероника',
                                           'Костя': 'Константин',
                                           'Колян': 'Николай',
                                           'Соня': 'Софья',
                                           'Лида': 'Лидия',
                                           'Данил': 'Даниил',
                                           'Данила': 'Даниил',
                                           'Аня': 'Анна',
                                           'Мари': 'Мария'
                                          }) 

In [29]:
len(df1.nickname.unique())

230

In [30]:
df1.nickname.unique()

array(['Владимир', 'Ольга', 'Виктория', 'unknown', 'Роман', 'Алина',
       'Дмитрий', 'Наталья', 'Екатерина', 'Дарья', 'Светлана', 'Арина',
       'Антон', 'Илья', 'Даниил', 'Гузель', 'Кирилл', 'Яна', 'Алена',
       'Джордж', 'Вячеслав', 'Алексей', 'Егор', 'Павел', 'Рузанна',
       'Никита', 'Александра', 'Лилия', 'Анастасия', 'Игорь', 'Олеся',
       'Рустам', 'Александр', 'Маргарита', 'Артем', 'Максим', 'Ирина',
       'Николай', 'Жанна', 'Юлия', 'Кристина', 'Ринат', 'Татьяна',
       'Евгений', 'Руслан', 'Виктор', 'Елена', 'Мария', 'Кумар', 'Сергей',
       'Венера', 'Юлиана', 'Ксения', 'Вероника', 'Валерия', 'Нина',
       'Вадим', 'Марина', 'Карина', 'Ерлан', 'Айгуль', 'Эльнора',
       'Альбек', 'Регина', 'Дина', 'Виолетта', 'Денис', 'Лунара',
       'Рузалия', 'Лиана', 'Азат', 'Ляззат', 'Саула', 'Вильдан',
       'Елизавета', 'Седа', 'Инга', 'Аманат', 'Валерий', 'Михаил',
       'Герман', 'Софья', 'Лидия', 'Розалия', 'Матвей', 'Вера', 'Инна',
       'Анна', 'Fernando', 'Рости

Неявные дубликаты столбца `nickname` убраны, было - 268 имён, стало - 230.

**df2**

In [31]:
# вспомним про данные
df2.head()

,date,source_income,orders,slot_duration,restaurant,income
0,15.12.2021,bk,16,12.0,Sovet,NaN
1,16.12.2021,bk,13,10.0,Sovet,NaN
2,17.12.2021,bk,13,12.0,Sovet,NaN
3,18.12.2021,bk,21,11.0,Sovet,NaN
4,19.12.2021,bk,13,12.0,Sovet,NaN


In [32]:
# изменим тип данных
df2['date'] = pd.to_datetime(df2['date'], format='%d.%m.%Y')

In [33]:
# посмотрим на изменения
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           174 non-null    datetime64[ns]
 1   source_income  174 non-null    object        
 2   orders         174 non-null    int64         
 3   slot_duration  174 non-null    float64       
 4   restaurant     67 non-null     object        
 5   income         113 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 8.3+ KB


Тип данных столбца `date` был изменем на "datetime".

In [34]:
# посмотрим где пропуски
df2[df2['income'].isna()]['source_income'].unique()

array(['bk'], dtype=object)

Ранее я говорил, что пропуски `income` только в строках `source_income` равное "bk". В подтверждение вывод выше.

Заполним пропуски расчётом значений.

In [35]:
# до изменений
df2[df2['income'].isna()][:5]

,date,source_income,orders,slot_duration,restaurant,income
0,2021-12-15,bk,16,12.0,Sovet,NaN
1,2021-12-16,bk,13,10.0,Sovet,NaN
2,2021-12-17,bk,13,12.0,Sovet,NaN
3,2021-12-18,bk,21,11.0,Sovet,NaN
4,2021-12-19,bk,13,12.0,Sovet,NaN


In [36]:
# заполним пропуски
df2['income'] = df2['income'].fillna(df2['orders'] * 150)
# посмотрим на результат
df2[df2['income'].isna()][:5]

,date,source_income,orders,slot_duration,restaurant,income


Как видно пропусков нет, а что же теперь там?

Посмотрим на строки со значением source_income == bk:

In [37]:
# результат замены
df2.query('source_income == "bk"').head()

,date,source_income,orders,slot_duration,restaurant,income
0,2021-12-15,bk,16,12.0,Sovet,2400.0
1,2021-12-16,bk,13,10.0,Sovet,1950.0
2,2021-12-17,bk,13,12.0,Sovet,1950.0
3,2021-12-18,bk,21,11.0,Sovet,3150.0
4,2021-12-19,bk,13,12.0,Sovet,1950.0


Заполним пропуски restaurant на фиксированное значение - unknown

In [38]:
# посмотрим на пропуски в restaurant
df2[df2['restaurant'].isna()][:5]

,date,source_income,orders,slot_duration,restaurant,income
67,2022-04-16,dc,7,5.0,NaN,1006.0
68,2022-04-17,dc,9,7.0,NaN,1411.0
69,2022-04-19,dc,9,8.0,NaN,1625.0
70,2022-04-20,dc,10,7.0,NaN,1905.0
71,2022-04-21,dc,5,6.0,NaN,1018.0


In [39]:
df2[df2['restaurant'].isna()]['source_income'].unique()

array(['dc'], dtype=object)

Видим, что пропуски в restaurant присутствуют только в source_income == dc. Причина в том, что в dc нет привязки к ресторану, заказы забираются с разных ресторанов. Заполним пропуски restaurant на фиксированное значение - unknown.

In [40]:
df2['restaurant'] = df2['restaurant'].fillna('unknown')
df2.query('source_income == "dc"').head()

,date,source_income,orders,slot_duration,restaurant,income
67,2022-04-16,dc,7,5.0,unknown,1006.0
68,2022-04-17,dc,9,7.0,unknown,1411.0
69,2022-04-19,dc,9,8.0,unknown,1625.0
70,2022-04-20,dc,10,7.0,unknown,1905.0
71,2022-04-21,dc,5,6.0,unknown,1018.0


In [41]:
# результат
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           174 non-null    datetime64[ns]
 1   source_income  174 non-null    object        
 2   orders         174 non-null    int64         
 3   slot_duration  174 non-null    float64       
 4   restaurant     174 non-null    object        
 5   income         174 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 8.3+ KB


Пропуски заполнены, типы данных приведены к нужным.

### Вывод

**df1**

* [X] Тип данных столбца `datetime` приведен к верному("datetime");
* [X] Убран неявный дубликат в `device_ios_or_android`;
* [X] Убраны неявные дубликаты имен в `nickname`. Было - 268, стало - 230.

Данные в **df1** подготовлены для исследовательского анализа данных.

**df2**

* [X] Тип данных столбца `date` приведен к верному("datetime");
* [X] Пропуски в `restaurant`, `income` были заполнены согласно специфике данных.

Данные в **df2** подготовлены для исследовательского анализа данных.

## EDA

**df1**

In [42]:
df1.describe()

,user_id,revenue
count,1.273000e+03,1273.000000
mean,7.941761e+10,885.215169
std,3.236076e+08,334.962393
min,7.900944e+10,455.990000
25%,7.917388e+10,665.000000
50%,7.922887e+10,764.000000
75%,7.986779e+10,1000.000000
max,7.999684e+10,3450.000000
